<a href="https://www.kaggle.com/code/felixfaruix/titanic-randomforest?scriptVersionId=250184056" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Titanic - Machine Learning from Disaster

This notebook is part of a _**Kaggle Competition**_ for the _Titanic_ dataset. The dataset is divided into train and test, where our goal is to predict what passengers survived the sank. We will do it using machine learning techniques and data science approach on the training data, building a model trying to predict the survivals on the test data. 

In [ ]:
"""Importing necessary libraries for the Titanic dataset analysis.
This script is part of a Kaggle competition notebook for the Titanic dataset."""

import pandas as pd 
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys

In [ ]:
# Load the Titanic dataset
# Ensure the correct path to the dataset files
# Adjust the path as necessary based on your directory structure
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

_Head()_ tells us the first 5 rows of the dataset, which is useful to understand the structure and content of the data.
As we can see, the dataset contains information about passengers such as their survival status, class, name and other features.     
This is useful to understand the structure and content of the data, and to identify any **potential issues or missing values** in the dataset.
We get a quick overview of the dataset, including the number of rows and columns, and the data types of each column.

More information can be obtained using the _info()_ and _describe()_ methods. Info method is particularly useful to understand the data types and missing values in the dataset, while describe() method gives us a statistical summary of the numerical columns.

In [ ]:
# First 5 rows of the training data
train_data.head()


In [ ]:
# Get a quick overview of the dataset
train_data.info()
train_data.describe()


# Parch shows the number of parents/children aboard, this one shows that most of the passengers were alone. 

The dataset is missing some values, especially in the '**Age**' and '**Embarked**' columns.
We can also see that the 'Survived' column is the target variable, which we will use to train our model.

 _Describe()_ shows that most of the people did not survive, which is expected as the Titanic sank.
The '**Pclass**' column indicates the class of the ticket, which is also an important feature to consider. This one shows that most of the passengers were in the 3rd class, which is expected as it was the cheapest ticket.
'**Parch**' column shows the number of parents/children aboard, pointing out that most of the passengers were alone. 

As our next step, we will now plot the correlation heatmap indicating the degree of linkage between each feature.


In [ ]:
# Converting Ticket column into a str, for better usage 
train_data["Ticket"] = train_data["Ticket"].astype(str)

In [ ]:
# Plotting only int or float columns correlations
numeric_df = train_data.select_dtypes(include=["int64", "float64"])
sns.heatmap(numeric_df.corr(), cmap='YlGnBu')
plt.title('Correlation Heatmap of Titanic Dataset')
plt.show()

#### Splitting our train set via stratified shuffle split from _sklearn_

Now we want to split our _train_data_ into train and test sets, but with preserved class proportions. 

- We shuffle our dataset randomly via **StratifiedShuffleSplit** from _sklearn_
- Then we split it into train and test sets (80%/20%)
- WHile preserving the distribution of different target variables in both sets (Survived, Pclass, Sex)

This is important when the class is imbalanced like in this case, where the survival rate is low. 


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)
for train_indeces, test_indeces in split.split(train_data, train_data[["Survived", "Pclass", "Sex"]]):
    strat_train_set = train_data.loc[train_indeces]
    strat_test_set = train_data.loc[test_indeces]

We plot each distirbution across the two sets, in order to verify that they are actually equal across each different variable. 

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 2, figsize=(10, 10))
fig.suptitle("Stratified Train/Test Distributions", fontsize=16)

# Row 1: Survived
axes[0, 0].hist(strat_train_set["Survived"], bins=2, color="skyblue")
axes[0, 0].set_title("Train: Survived")
axes[0, 1].hist(strat_test_set["Survived"], bins=2, color="brown")
axes[0, 1].set_title("Test: Survived")

# Row 2: Sex
axes[1, 0].hist(strat_train_set["Sex"], bins=2, color="skyblue")
axes[1, 0].set_title("Train: Sex")
axes[1, 1].hist(strat_test_set["Sex"], bins=2, color="brown")
axes[1, 1].set_title("Test: Sex")

# Row 3: Pclass
axes[2, 0].hist(strat_train_set["Pclass"], bins=3, color="skyblue")
axes[2, 0].set_title("Train: Pclass")
axes[2, 1].hist(strat_test_set["Pclass"], bins=3, color="brown")
axes[2, 1].set_title("Test: Pclass")

# Clean up
for ax in axes.flat:
    ax.set_xticks([0, 1, 2, 3])
    ax.set_xlabel("")
    ax.set_ylabel("Count")

plt.tight_layout(pad = 0.8, h_pad= 0.2, w_pad= 0.2)
plt.show()

In [ ]:
strat_train_set.info()

### Feature Engineering 

For first, we need to fill the gaps in the Age column. 
Then, we will have to drop the name column, which is a string, not brining any value to our feature space. Then, we will also transform our '**Sex**' column into a binary variable (0: Male - 1: Female). 

We will also drop '**Cabin**' column and transform into integers the values from the '**Embarked**' column 

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin


class FeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols=("Name", "Ticket", "Cabin")):
        self.cols = list(cols)
    def fit(self, X, y=None): return self
    def transform(self, X):
        return X.drop(columns=self.cols, errors="ignore").copy()

class AgeImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imp = SimpleImputer(strategy="mean")
    def fit(self, X, y=None):
        self.imp.fit(X[["Age"]]);  return self
    def transform(self, X):
        X = X.copy()
        age_imputed = self.imp.transform(X[["Age"]])
        X["Age"] = age_imputed.ravel() 
        return X

class CategoricalMapper(BaseEstimator, TransformerMixin):
    sex_map = {"male": 0, "female": 1, "0": 0, "1": 1}
    emb_map = {"C": 1, "Q": 2, "S": 3, "N": 0,
               "1": 1, "2": 2, "3": 3, "0": 0}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # --- Sex (works if already int or str)
        X["Sex"] = (
            X["Sex"]
            .astype(str)
            .str.lower()
            .map(self.sex_map)
            .astype("Int64")
        )

        # --- Embarked (works if already int or str / NaN)
        X["Embarked"] = (
            X["Embarked"]
              .fillna("N")
              .astype(str)
              .str.upper()
              .map(self.emb_map)
              .astype("Int64")
        )
        return X


In [ ]:
from sklearn.pipeline import Pipeline

preprocess_pipe = Pipeline([
    ("drop", FeatureDropper()),
    ("age",  AgeImputer()),
    ("cat",  CategoricalMapper()),
])
print("TRAIN cols :", list(strat_train_set.columns))
print("VAL   cols :", list(strat_test_set.columns))
# fit + transform on training split
strat_train_set = preprocess_pipe.fit_transform(strat_train_set)

# transform ONLY on validation / test split
strat_test_set  = preprocess_pipe.transform(strat_test_set)

In [ ]:
strat_train_set.info()

In [ ]:
strat_test_set.info()

We notice that the dataset is clean now. All the features have the same number of values. 

#### Training and Evaluation of the Model (Random Forest Classifier)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

X_train = strat_train_set.drop("Survived", axis=1)
y_train = strat_train_set["Survived"]
X_val = strat_test_set.drop("Survived", axis=1)
y_val = strat_test_set["Survived"]

# Define the model
rf_model = RandomForestClassifier(random_state=42)
# Define the parameter grid for GridSearchCV
param_grid = [
    {'n_estimators': [10, 100, 200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [3, 5, 10],
    }]
grid_search = GridSearchCV(rf_model, param_grid, cv=3, scoring='accuracy', return_train_score=True) 
grid_search.fit(X_train, y_train)

In [ ]:
best_rf_model = grid_search.best_estimator_
val_accuracy = best_rf_model.score(X_val, y_val)

We are now plotting the scores: 

In [ ]:
print("Best cross-validated accuracy (train folds):", grid_search.best_score_)
print("Final accuracy on hold-out validation set (strat_test_set):", val_accuracy)
print("Best parameters:", grid_search.best_params_)

In [ ]:
test_data.info()

In [ ]:
# Fill forward missing Fare values in test_data
test_data["Fare"] = test_data["Fare"].ffill()
test_data["Fare"].isna().sum()  # Should be 0 if all missing values are filled

In [ ]:
test_data = preprocess_pipe.transform(test_data)

In [ ]:
test_data.info()

In [ ]:
print("len(test_data)      :", len(test_data))        # should be 418


In [ ]:
test_predictions = best_rf_model.predict(test_data)
print("len(test_predictions):", len(test_predictions))# must be 418

In [ ]:
test_predictions = best_rf_model.predict(test_data)

submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})
submission.to_csv("submission.csv", index=False, mode="w")

In [ ]:
check = pd.read_csv("submission.csv")
print("rows on disk      :", len(check))   